# Binary Classification

In [14]:
# Import Library
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### 1. Data (Preparing and Loading)

In [15]:
# Load dataset
df = pd.read_csv("C:/Users/arya.hisma/Downloads/000. UDEMY/Udemy - Pytorch/Logistic Regression Using Pytorch - Customer Churn/dataset/customer_churn_dataset_training_master.csv")

# Display data
df.head(5)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [16]:
df.shape

(440833, 12)

In [17]:
'''
Notes :
1. CustomerID → ID unik pelanggan (sering tidak diperlukan untuk analisis).
2. Age → Usia pelanggan (numerik).
3. Gender → Jenis kelamin pelanggan (kategori: 'Male', 'Female').
4. Tenure → Lama pelanggan berlangganan (numerik, biasanya dalam bulan/tahun).
5. Usage Frequency → Seberapa sering pelanggan menggunakan layanan (numerik).
6. Support Calls → Jumlah panggilan ke customer support (numerik).
7. Payment Delay → Keterlambatan pembayaran (numerik, bisa dalam hari).
8. Subscription Type → Jenis langganan (kategori: 'Basic', 'Premium', dll.).
9. Contract Length → Durasi kontrak pelanggan (numerik atau kategori).
10. Total Spend → Total uang yang dihabiskan pelanggan (numerik).
11. Last Interaction → Waktu terakhir pelanggan berinteraksi dengan layanan (bisa berupa tanggal atau numerik).
12. Churn → Apakah pelanggan berhenti berlangganan (biner: 'Yes' atau 'No').
'''

"\nNotes :\n1. CustomerID → ID unik pelanggan (sering tidak diperlukan untuk analisis).\n2. Age → Usia pelanggan (numerik).\n3. Gender → Jenis kelamin pelanggan (kategori: 'Male', 'Female').\n4. Tenure → Lama pelanggan berlangganan (numerik, biasanya dalam bulan/tahun).\n5. Usage Frequency → Seberapa sering pelanggan menggunakan layanan (numerik).\n6. Support Calls → Jumlah panggilan ke customer support (numerik).\n7. Payment Delay → Keterlambatan pembayaran (numerik, bisa dalam hari).\n8. Subscription Type → Jenis langganan (kategori: 'Basic', 'Premium', dll.).\n9. Contract Length → Durasi kontrak pelanggan (numerik atau kategori).\n10. Total Spend → Total uang yang dihabiskan pelanggan (numerik).\n11. Last Interaction → Waktu terakhir pelanggan berinteraksi dengan layanan (bisa berupa tanggal atau numerik).\n12. Churn → Apakah pelanggan berhenti berlangganan (biner: 'Yes' atau 'No').\n"

In [18]:
df.isnull().sum()

CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64

In [19]:
# Hapus missing value

df = df.dropna()
df.shape

(440832, 12)

In [20]:
# Remove unnecessary columns
df_clean = df.drop(["CustomerID"], axis=1)
df_clean.head(5)

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [21]:
# Data Info
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440832 entries, 0 to 440832
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Age                440832 non-null  float64
 1   Gender             440832 non-null  object 
 2   Tenure             440832 non-null  float64
 3   Usage Frequency    440832 non-null  float64
 4   Support Calls      440832 non-null  float64
 5   Payment Delay      440832 non-null  float64
 6   Subscription Type  440832 non-null  object 
 7   Contract Length    440832 non-null  object 
 8   Total Spend        440832 non-null  float64
 9   Last Interaction   440832 non-null  float64
 10  Churn              440832 non-null  float64
dtypes: float64(8), object(3)
memory usage: 40.4+ MB


In [22]:
# Tentukan fitur dan label
X = df_clean.drop('Churn', axis=1)
y = df_clean['Churn']

X

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0
...,...,...,...,...,...,...,...,...,...,...
440828,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0
440829,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0
440830,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0
440831,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0


In [23]:
# Pisahkan kolom numerik dan kategorikal
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [ ]:
numerical_cols, categorical_cols

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

# Buat Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_cols), # Scaling kolom numerik
        ('cat', OneHotEncoder(drop='first'), categorical_cols) # OneHot kolom kategorikal
    ]
)

In [ ]:
# Buat Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
pipeline

In [ ]:
# Split Train Test Split
X_train, X_test, y_train, y_train = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Transform X_train dan X_test
X_train_processed = pipeline.fit_transform(X_train)  
X_test_processed = pipeline.transform(X_test)

In [ ]:
X_train_processed

In [ ]:
X_test_processed

In [ ]:
# Ambil nama kolom hasil one hot encoder dan scaling
ohe = pipeline.named_steps['preprocessor'].named_transformers_['cat']
encoded_col_names = ohe.get_feature_names_out(categorical_cols)
encoded_col_names

In [ ]:
# Gabungkan semua nama kolom : fitur numerik + fitur kategorikal encoded
final_feature_names = list(numerical_cols) + list(encoded_col_names)
final_feature_names

In [ ]:
X_train_processed[0]

In [ ]:
# Tentukan Kolom
numerical_cols = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay',
                  'Total Spend', 'Last Interaction']
categorical_onehot_cols = ['Gender']
categorical_ordinal_cols = ['Subscription Type', 'Contract Length']

In [ ]:
# Buat Transformer
preprocessor = ColumnTransformer(
    transformers = [
        ('num', MinMaxScaler(), numerical_cols),
        ('onehot', OneHotEncoder(drop='first'), categorical_onehot_cols),
        ('ordinal', OrdinalEncoder(), categorical_ordinal_cols)
    ]
)

In [ ]:
# Buat Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
pipeline

In [ ]:
# Split Data Train Test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=X['Gender'])

In [ ]:
X_train.shape

In [ ]:
# Fit Transform
X_train_processed = pipeline.fit_transform(X_train)
X_test_processed = pipeline.transform(X_test)

In [ ]:
X_train.head(1)

In [ ]:
X_train_processed[0]

In [ ]:
# Ambil Transformers
ohe = pipeline.named_steps['preprocessor'].named_transformers_['onehot']
ordinal = pipeline.named_steps['preprocessor'].named_transformers_['ordinal']

ohe, ordinal

In [ ]:
# Ambil Nama Kolom Hasil One Hot Encoder
onehot_feature_names = ohe.get_feature_names_out(categorical_onehot_cols)

# Gabungkan Semua Kolom
final_feature_names = (
    numerical_cols +
    list(onehot_feature_names) +
    categorical_ordinal_cols
)

final_feature_names

In [ ]:
# Cek Hasil Encoding
# Ambil Encodernya
ordinal_encoder = pipeline.named_steps['preprocessor'].named_transformers_['ordinal']
print(ordinal_encoder.categories_)
categories = ordinal_encoder.categories_

# Gabungkan jadi dict nama kolom dan kategorinya
ordinal_col_info = dict(zip(categorical_ordinal_cols, categories))

for col, cat in ordinal_col_info.items():
    print(f"{col}: {list(cat)}")

# Tampilkan Hasil Mapping Kategori ke Angka
for cols, cats in zip(categorical_ordinal_cols, ordinal_encoder.categories_):
    print(f"Kolom {cols} kategori : angka")
    
    for i, cat in enumerate(cats):
        print(f"{cat} : {i}")

In [ ]:
# Data Yang Dipakai
X_train_processed.shape, X_test_processed.shape, y_train.shape, y_test.shape

In [ ]:
type(X_train_processed), type(X_test_processed), type(y_train), type(y_test)

In [ ]:
# Create model
# konversi ke tensor pytorch (karena format sebelumnya adalah dataframe, maka diganti terlebih dahulu ke numpy)
X_train_tensor = torch.tensor(X_train_processed,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_processed,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(dtype="float32")).unsqueeze(dim=1)
y_test_tensor = torch.tensor(y_test.to_numpy(dtype="float32")).unsqueeze(dim=1)

# X_train_tensor = torch.tensor(X_train)
# X_test_tensor = torch.tensor(X_test)
# y_train_tensor = torch.tensor(y_train).unsqueeze(dim=1)
# y_test_tensor = torch.tensor(y_test).unsqueeze(dim=1)

X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape

In [ ]:
y_train_tensor

In [ ]:
X_train_tensor.shape[1]

In [ ]:
# LR Model 1

# class LRModel (nn.Module):
#     def __init__(self, input_dim):
#         super(LRModel, self).__init__()
#         self.linear = nn.Linear(input_dim, 1)
        
#     def forward(self, x):
#         return torch.sigmoid(self.linear(x))

input_dim = X_train_tensor.shape[1]

model1 = nn.Sequential(
    nn.Linear(input_dim, 1),
    nn.Sigmoid()
)

model1 

In [ ]:
# Loss & Optimize

criterion = nn.BCELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.01)

In [ ]:
# Training Loop
torch.manual_seed(42)

epochs = 1000
losses_train = []
losses_test = []

for epoch in range(epochs):
    model1.train()
    
    output_train = model1(X_train_tensor)

    loss_train = criterion(output_train,  y_train_tensor)
    
    losses_train.append(loss_train.item())
    
    output_label_train = (output_train >= 0.5).float()
        
    accuracy_train = (output_label_train == y_train_tensor).float().mean()
        
    optimizer.zero_grad()
    
    loss_train.backward()
    
    optimizer.step()
    
    # Evaluasi akurasi
    from sklearn.metrics import confusion_matrix, classification_report

    model1.eval()
    with torch.no_grad():
        output_test = model1(X_test_tensor)
        
        loss_test = criterion(output_test,  y_test_tensor)
        
        losses_test.append(loss_test.item())
        
        output_label_test = (output_test >= 0.5).float()
                
        accuracy_test = (output_label_test == y_test_tensor).float().mean()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch : [{epoch+1}/{epochs}] | Loss_train : {loss_train.item():.4f} | Loss_test : {loss_test.item():.4f} | Accuracy_train : {accuracy_train:.4f}  | Accuracy Test : {accuracy_test:.4f}")

In [ ]:
# konversi ke numpy untuk sklearn
y_true_train = y_train_tensor.numpy()
y_pred_train = output_label_train.numpy()

# Confusion matrix data train
cm_train = confusion_matrix(y_true_train, y_pred_train)
print(f"confusion matrix : {cm_train}")

# Classification report (precision, recall, f1-score) data train
cr_train = classification_report(y_true_train, y_pred_train)
print(f"Classification report : {cr_train}")

In [ ]:
# konversi ke numpy untuk sklearn
y_true_test = y_test_tensor.numpy()
y_pred_test = output_label_test.numpy()
        
# Confusion matrix data test
cm_test = confusion_matrix(y_true_test, y_pred_test)
print(f"confusion matrix : {cm_test}")

# Classification report (precision, recall, f1-score) data test
cr_test = classification_report(y_true_test, y_pred_test)
print(f"Classification report : {cr_test}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Visualisasi
plt.figure(figsize=(6,4))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Visualisasi
plt.figure(figsize=(6,4))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
type(output_label_test)

In [ ]:
type(y_test_tensor)

In [ ]:
print(losses_train)
print(losses_test)

In [ ]:
# Plot loss per epoch
plt.plot(range(epochs), losses_train, label='Train Loss')
plt.plot(range(epochs), losses_test, label='Test Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# class LRModel2 (nn.Module):
#     def __init__(self, input_dim):
#         super(LRModel2, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 16)
#         self.relu = nn.ReLU()
#         self.output = nn.Linear(16, 1)
        
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.output(x)
#         return torch.sigmoid(x)

model2 = nn.Sequential(
    nn.Linear(input_dim, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

model2

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.01)

In [ ]:
# Training Loop
torch.manual_seed(42)

epochs = 1000
losses_train = []
losses_test = []

for epoch in range(epochs):
    model2.train()
    
    output_train = model2(X_train_tensor)

    loss_train = criterion(output_train,  y_train_tensor)
    
    losses_train.append(loss_train.item())
    
    output_label_train = (output_train >= 0.5).float()
        
    accuracy_train = (output_label_train == y_train_tensor).float().mean()
      
    optimizer.zero_grad()
    
    loss_train.backward()
    
    optimizer.step()
    
    # Evaluasi akurasi
    from sklearn.metrics import confusion_matrix, classification_report

    model2.eval()
    with torch.no_grad():
        output_test = model2(X_test_tensor)
        
        loss_test = criterion(output_test,  y_test_tensor)
        
        losses_test.append(loss_test.item())
        
        output_label_test = (output_test >= 0.5).float()
                
        accuracy_test = (output_label_test == y_test_tensor).float().mean()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch : [{epoch+1}/{epochs}] | Loss_train : {loss_train.item():.4f} | Loss_test : {loss_test.item():.4f} | Accuracy_train : {accuracy_train:.4f}  | Accuracy Test : {accuracy_test:.4f}")

In [ ]:
# konversi ke numpy untuk sklearn
y_true_train = y_train_tensor.numpy()
y_pred_train = output_label_train.numpy()
    
# Confusion matrix data train
cm_train = confusion_matrix(y_true_train, y_pred_train)
print(f"confusion matrix : {cm_train}")

# Classification report (precision, recall, f1-score) data train
cr_train = classification_report(y_true_train, y_pred_train)
print(f"Classification report : {cr_train}")

In [ ]:
# konversi ke numpy untuk sklearn
y_true_test = y_test_tensor.numpy()
y_pred_test = output_label_test.numpy()
        
# Confusion matrix data test
cm_test = confusion_matrix(y_true_test, y_pred_test)
print(f"confusion matrix : {cm_test}")

# Classification report (precision, recall, f1-score) data test
cr_test = classification_report(y_true_test, y_pred_test)
print(f"Classification report : {cr_test}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Visualisasi
plt.figure(figsize=(6,4))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Visualisasi
plt.figure(figsize=(6,4))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(losses_train)
print(losses_test)

In [ ]:
# Plot loss per epoch
plt.plot(range(epochs), losses_train, label='Train Loss')
plt.plot(range(epochs), losses_test, label='Test Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# class LRModel3 (nn.Module):
#     def __init__(self, input_dim):
#         super(LRModel3, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 32)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(32, 16)
#         self.relu2 = nn.ReLU()
#         self.output = nn.Linear(16, 1)
        
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu1(x)
#         x = self.fc2(x)
#         x = self.relu2(x)
#         x = self.output(x)
#         return torch.sigmoid(x)


model3 = nn.Sequential(
    nn.Linear(input_dim, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

model3

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model3.parameters(), lr=0.01)

In [ ]:
# Training Loop
torch.manual_seed(42)

epochs = 1000
losses_train = []
losses_test = []

for epoch in range(epochs):
    model3.train()
    
    output_train = model3(X_train_tensor)

    loss_train = criterion(output_train,  y_train_tensor)
    
    losses_train.append(loss_train.item())
    
    output_label_train = (output_train >= 0.5).float()
        
    accuracy_train = (output_label_train == y_train_tensor).float().mean()
       
    optimizer.zero_grad()
    
    loss_train.backward()
    
    optimizer.step()
    
    # Evaluasi akurasi
    from sklearn.metrics import confusion_matrix, classification_report

    model3.eval()
    with torch.no_grad():
        output_test = model3(X_test_tensor)
        
        loss_test = criterion(output_test,  y_test_tensor)
        
        losses_test.append(loss_test.item())
        
        output_label_test = (output_test >= 0.5).float()
                
        accuracy_test = (output_label_test == y_test_tensor).float().mean()
      
    if (epoch + 1) % 10 == 0:
        print(f"Epoch : [{epoch+1}/{epochs}] | Loss_train : {loss_train.item():.4f} | Loss_test : {loss_test.item():.4f} | Accuracy_train : {accuracy_train:.4f}  | Accuracy Test : {accuracy_test:.4f}")

In [ ]:
# konversi ke numpy untuk sklearn
y_true_train = y_train_tensor.numpy()
y_pred_train = output_label_train.numpy()
    
# Confusion matrix data train
cm_train = confusion_matrix(y_true_train, y_pred_train)
print(f"confusion matrix : {cm_train}")

# Classification report (precision, recall, f1-score) data train
cr_train = classification_report(y_true_train, y_pred_train)
print(f"Classification report : {cr_train}")

In [ ]:
# konversi ke numpy untuk sklearn
y_true_test = y_test_tensor.numpy()
y_pred_test = output_label_test.numpy()

# Confusion matrix data test
cm_test = confusion_matrix(y_true_test, y_pred_test)
print(f"confusion matrix : {cm_test}")

# Classification report (precision, recall, f1-score) data test
cr_test = classification_report(y_true_test, y_pred_test)
print(f"Classification report : {cr_test}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Visualisasi
plt.figure(figsize=(6,4))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Visualisasi
plt.figure(figsize=(6,4))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(losses_train)
print(losses_test)

In [ ]:
# Plot loss per epoch
plt.plot(range(epochs), losses_train, label='Train Loss')
plt.plot(range(epochs), losses_test, label='Test Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Simpan Model

import torch

torch.save(model3, 'app/model/model_churn3.pth')

In [ ]:
# Simpan Pipeline
import joblib

joblib.dump(pipeline, 'app/model/preprocessing_pipeline.pkl')

In [ ]:
input_data = {
    'Age' : [30],
    'Gender' : ['Female'],
    'Tenure' : [39],
    'Usage Frequency' : [14],
    'Support Calls' : [5],
    'Payment Delay' : [18],
    'Subscription Type' : ['Standard'],
    'Contract Length' : ['Annual'],
    'Total Spend' : [932],
    'Last Interaction' : [17]
}

input_df = pd.DataFrame(input_data)
input_df

In [ ]:
# Load Preprocessed Data
pipeline_loaded = joblib.load('app/model/preprocessing_pipeline.pkl')

In [ ]:
# Transform Input Data
input_processed = pipeline_loaded.transform(input_df)
input_processed

In [ ]:
# Ubah menjadi tensor
input_tensor = torch.tensor(input_processed, dtype=torch.float32)
input_tensor

In [ ]:
# Prediksi dengan model
# Load Model
model3 = torch.load('app/model/model_churn3.pth', weights_only=False)
model3

In [ ]:
# Prediksi
with torch.no_grad():
    output = model3(input_tensor)

print(f"Output : {output}") 

pred_prob = output.item()
pred_label = 1 if pred_prob >= 0.5 else 0

print(f"Probabilitas churn: {pred_prob:.4f}")
print(f"Prediksi: {'Churn' if pred_label == 1 else 'Tidak Churn'}")

In [ ]:
from torchinfo import summary

summary(model3)

In [ ]:
from torchinfo import summary

summary(model2)

In [ ]:
from torchinfo import summary

summary(model1)